# Weighted Temporal Matrix Factorization (WTMF)

### Model Description

The optimization problem of WTMF is given by

\begin{equation}
\begin{aligned}
\min_{\boldsymbol{W},\boldsymbol{X},\boldsymbol{A}}~&\frac{1}{2}\|\mathcal{P}_{\Omega}(\boldsymbol{Y}-\boldsymbol{W}^\top\boldsymbol{X})\|_{F}^{2}+\rho\sum_{r=1}^{R}(\|\boldsymbol{w}_{r}\|_{2}^{2}+\|\boldsymbol{x}_{r}\|_{2}^{2}+\eta^2)^{p/2} \\
&+\frac{\lambda}{2}\|\boldsymbol{X}\boldsymbol{\Psi}_{0}^\top-\boldsymbol{A}(\boldsymbol{I}_{d}\otimes\boldsymbol{X})\boldsymbol{\Psi}^\top\|_{F}^{2} \\
\end{aligned}
\end{equation}

### Model Solution

There is an alternating minimization scheme for solving this problem:

\begin{equation}
\begin{cases}
\boldsymbol{W}:=\{\boldsymbol{W}\,\mid\,\boldsymbol{X}\mathcal{P}_{\Omega}^\top(\boldsymbol{W}^\top\boldsymbol{X})+\rho D_{(\boldsymbol{W},\boldsymbol{X})}\boldsymbol{W}=\boldsymbol{X}\mathcal{P}_{\Omega}^\top(\boldsymbol{Y})\} \\
\boldsymbol{X}:=\{\boldsymbol{X}\,\mid\,\boldsymbol{W}\mathcal{P}_{\Omega}(\boldsymbol{W}^\top\boldsymbol{X})+\rho D_{(\boldsymbol{W},\boldsymbol{X})}\boldsymbol{X}+\lambda\sum_{k=0}^{d}\boldsymbol{A}_{k}^\top\left(\sum_{h=0}^{d}\boldsymbol{A}_{h}\boldsymbol{X}\boldsymbol{\Psi}_{h}^\top\right)\boldsymbol{\Psi}_{k}=\boldsymbol{W}\mathcal{P}_{\Omega}(\boldsymbol{Y})\} \\
\boldsymbol{A}:=\boldsymbol{X}\boldsymbol{\Psi}_{0}^\top[(\boldsymbol{I}_{d}\otimes\boldsymbol{X})\boldsymbol{\Psi}^\top]^\dagger \\
\end{cases}
\end{equation}
where the matrix $D_{(\boldsymbol{W},\boldsymbol{X})}\in\mathbb{R}^{R\times R}$ is defined as follows,

\begin{equation}
\begin{aligned}
D_{(\boldsymbol{W},\boldsymbol{X})}=p\cdot\text{diag}\big(&(\|\boldsymbol{w}_{1}\|_{2}^{2}+\|\boldsymbol{x}_{1}\|_{2}^{2}+\eta^2)^{p/2-1},\ldots,(\|\boldsymbol{w}_{R}\|_{2}^{2}+\|\boldsymbol{x}_{R}\|_{2}^{2}+\eta^2)^{p/2-1}\big)
\end{aligned}
\end{equation}

This model refers to:

> Giampouras, P. V., Rontogiannis, A. A., & Koutroumbas, K. D. (2018). Alternating iteratively reweighted least squares minimization for low-rank matrix factorization. IEEE Transactions on Signal Processing, 67(2), 490-503.

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def generate_Psi(T, d, season):
    Psi = []
    for k in range(0, d + 1):
        if k == 0:
            Psi.append(np.append(np.zeros((T - d - season, d)), 
                                 np.append(-1 * np.eye(T - d - season), np.zeros((T - d - season, season)), axis = 1) 
                                 + np.append(np.zeros((T - d - season, season)), np.eye(T - d - season), axis = 1), axis = 1))
        else:
            Psi.append(np.append(np.append(np.zeros((T - d - season, d - k)), 
                                           np.append(-1 * np.eye(T - d - season), np.zeros((T - d - season, season)), axis = 1)
                                           + np.append(np.zeros((T - d - season, season)), np.eye(T - d - season), axis = 1), axis = 1), 
                                 np.zeros((T - d - season, k)), axis = 1))
    return Psi

def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew

def ell_w(ind, W, X, D, rho):
    return X @ ((W.T @ X) * ind).T + rho * D @ W

def conj_grad_w(sparse_mat, ind, W, X, D, rho, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X, D, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X, D, rho), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(ind, W, X, D, A, Psi, d, lmbda, rho):
    rank, dim2 = X.shape
    temp = np.zeros((d * rank, Psi[0].shape[0]))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    temp1 = X @ Psi[0].T - A @ temp
    temp2 = np.zeros((rank, dim2))
    for k in range(d):
        temp2 += A[:, k * rank : (k + 1) * rank].T @ temp1 @ Psi[k + 1]
    return W @ ((W.T @ X) * ind) + rho * D @ X + lmbda * (temp1 @ Psi[0] - temp2)

def conj_grad_x(sparse_mat, ind, W, X, D, A, Psi, d, lmbda, rho, maxiter = 5):
    rank, dim2 = X.shape
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat - ell_x(ind, W, X, D, A, Psi, d, lmbda, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(ind, W, Q, D, A, Psi, d, lmbda, rho), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

def wtmf(dense_mat, sparse_mat, rank, d, lmbda, rho, p, eta, season, maxiter):
    dim1, dim2 = sparse_mat.shape
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    A = 0.01 * np.random.randn(rank, d * rank)
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    Psi = generate_Psi(dim2, d, season)
    show_iter = 50
    temp = np.zeros((d * rank, dim2 - d - season))
    for it in range(maxiter):
        D = np.eye(rank)
        for r in range(rank):
            D[r, r] = p * np.power(np.linalg.norm(W[r, :], 2) ** 2
                                   + np.linalg.norm(X[r, :], 2) ** 2 + eta ** 2, p / 2 - 1)
        W = conj_grad_w(sparse_mat, ind, W, X, D, rho)
        for r in range(rank):
            D[r, r] = p * np.power(np.linalg.norm(W[r, :], 2) ** 2
                                   + np.linalg.norm(X[r, :], 2) ** 2 + eta ** 2, p / 2 - 1)
        X = conj_grad_x(sparse_mat, ind, W, X, D, A, Psi, d, lmbda, rho)
        for k in range(1, d + 1):
            temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
        A = X @ Psi[0].T @ np.linalg.pinv(temp)
        mat_hat = W.T @ X
        if (it + 1) % show_iter == 0:
            temp_hat = mat_hat[pos_test]
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return mat_hat, W, X, A

def wtmf_dic(obs, W, X, A, d, lmbda, rho, p, eta, season):
    dim1, dim2 = obs.shape
    rank = X.shape[0]
    if np.isnan(obs).any() == False:
        ind = obs != 0
    elif np.isnan(obs).any() == True:
        ind = ~np.isnan(obs)
        obs[np.isnan(obs)] = 0
    Psi = generate_Psi(dim2, d, season)
    D = np.eye(rank)
    for r in range(rank):
        D[r, r] = p * np.power(np.linalg.norm(W[r, :], 2) ** 2
                                + np.linalg.norm(X[r, :], 2) ** 2 + eta ** 2, p / 2 - 1)
    X = conj_grad_x(obs, ind, W, X, D, A, Psi, d, lmbda, rho)
    temp = np.zeros((d * rank, dim2 - d - season))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    A = X @ Psi[0].T @ np.linalg.pinv(temp)
    return X, A

def var4cast(X, A, d, delta, season):
    dim1, dim2 = X.shape
    X_hat = np.append(X[:, season:] - X[:, : -season], np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X_hat[:, dim2 - season + t] = A @ X_hat[:, dim2 - season + t - np.arange(1, d + 1)].T.reshape(dim1 * d)
    X = np.append(X, np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X[:, dim2 + t] = X[:, dim2 - season + t] + X_hat[:, dim2 - season + t]
    return X

from ipywidgets import IntProgress
from IPython.display import display

def rolling4cast(dense_mat, sparse_mat, pred_step, delta, rank, d, lmbda, rho, p, eta, season, maxiter):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / delta))
    mat_hat = np.zeros((dim1, max_count * delta))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            _, W, X, A = wtmf(dense_mat[:, : start_time], sparse_mat[:, : start_time], 
                              rank, d, lmbda, rho, p, eta, season, maxiter)
        else:
            X, A = wtmf_dic(sparse_mat[:, : start_time + t * delta], W, X_new, 
                            A, d, lmbda, rho, p, eta, season)
        X_new = var4cast(X, A, d, delta, season)
        mat_hat[:, t * delta : (t + 1) * delta] = W.T @ X_new[:, - delta :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where((small_dense_mat != 0) & (np.invert(np.isnan(small_dense_mat))))
    mape = compute_mape(small_dense_mat[pos], mat_hat[pos])
    rmse = compute_rmse(small_dense_mat[pos], mat_hat[pos])
    print('Prediction MAPE: {:.6}'.format(mape))
    print('Prediction RMSE: {:.6}'.format(rmse))
    print()
    return mat_hat, W, X, A

In [ ]:
import numpy as np

dense_mat = np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
sparse_mat = np.round(np.random.rand(dense_mat.shape[0], dense_mat.shape[1])) * dense_mat

import time
for rank in [10]: # rank of matrix factorization
    for delta in [1, 2, 3, 6]: # forecasting time horizon
        for d in [6]: # order of VAR
            start = time.time()
            pred_step = 7 * 24
            lmbda = 1
            rho =  5
            p = 0.5
            eta = 1
            season = 7 * 24
            maxiter = 50
            mat_hat, W, X, A = rolling4cast(dense_mat[:, : 24 * 7 * 10], dense_mat[:, : 24 * 7 * 10], 
                                            pred_step, delta, rank, d, lmbda, rho, p, eta, season, maxiter)
            print('delta = {}'.format(delta))
            print('rank R = {}'.format(rank))
            print('Order d = {}'.format(d))
            end = time.time()
            print('Running time: %d seconds'%(end - start))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>